# [6] Wardバッチ処理 - Phase 3

ミニマップキャプチャからward座標を抽出し、タイムラインデータと統合します。

## 処理フロー
1. **YOLO推論**: 全フレームでward検出（GPUバッチ推論対応）
2. **クラスタリング**: 同一wardをグループ化
3. **タイムライン統合**: Riot APIのwardイベントとマッチング

## 入出力
- **入力**: ミニマップ画像 (`C:\dataset_20260105\JP1-*\0\*.png`)
- **出力**: 
  - `detections_raw.csv` - 生の検出結果
  - `wards.csv` - クラスタリング後のward情報
  - `wards_matched.csv` - タイムライン統合済み

## 前提条件
- Phase 2完了（YOLOv8モデル `models/best.pt`）
- ミニマップキャプチャ完了（notebook 04）
- タイムラインデータ（`data/timeline/*.json`）

In [1]:
#cell-1: ライブラリインポート
from pathlib import Path
import numpy as np
import pandas as pd
import csv
from collections import defaultdict
from dataclasses import dataclass, field
from typing import List, Dict
import time

from tqdm import tqdm
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import torch

PROJECT_ROOT = Path(r"c:\Users\lapis\Desktop\LoL_WorkSp_win\pyLoL-_WorkSp\pyLoL-v2")

# autoLeagueモジュール
import sys
sys.path.insert(0, str(PROJECT_ROOT))

print("インポート完了")

インポート完了


In [2]:
#cell-2: パス・設定

# モデルパス
MODEL_PATH = PROJECT_ROOT / "models" / "best.pt"

# データセットパス
DATASET_DIR = Path(r"C:\dataset_20260105")

# タイムラインデータ
TIMELINE_DIR = PROJECT_ROOT / "data" / "timeline"

# 推論設定
CONFIDENCE_THRESHOLD = 0.6
IMAGE_SIZE = 512
BATCH_SIZE = 8  # GPUバッチ推論サイズ（VRAM 8GB以上推奨）

# クラスタリング設定
DISTANCE_THRESHOLD = 0.01  # 同一wardと判定する座標距離（正規化座標、約5px）
MIN_FRAMES = 3  # ノイズ除去：最小連続フレーム数
GAP_TOLERANCE = 10  # 検出が途切れても同一wardとみなすフレーム数

print(f"モデル: {MODEL_PATH}")
print(f"  存在: {MODEL_PATH.exists()}")
print(f"\nデータセット: {DATASET_DIR}")
print(f"  存在: {DATASET_DIR.exists()}")

if DATASET_DIR.exists():
    match_dirs = sorted(DATASET_DIR.glob("JP1-*"))
    print(f"  試合数: {len(match_dirs)}")

# GPU確認
if torch.cuda.is_available():
    print(f"\nGPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    print(f"バッチサイズ: {BATCH_SIZE}")
else:
    print("\nGPU: 利用不可（CPUで実行）")
    BATCH_SIZE = 1

モデル: c:\Users\lapis\Desktop\LoL_WorkSp_win\pyLoL-_WorkSp\pyLoL-v2\models\best.pt
  存在: True

データセット: C:\dataset_20260105
  存在: True
  試合数: 123

GPU: NVIDIA GeForce RTX 3050 Laptop GPU
VRAM: 4.0 GB
バッチサイズ: 8


In [3]:
#cell-3: YOLOモデル読み込み

model = YOLO(str(MODEL_PATH))
print(f"モデル読み込み完了")
print(f"クラス: {model.names}")

モデル読み込み完了
クラス: {0: 'stealth_ward', 1: 'stealth_ward_enemy', 2: 'control_ward', 3: 'control_ward_enemy'}


## データ構造定義

In [4]:
#cell-4: データ構造定義

@dataclass
class Detection:
    """1フレームでの検出結果"""
    frame: int
    class_id: int
    class_name: str
    x: float  # 正規化座標 (0-1)
    y: float
    w: float
    h: float
    confidence: float


@dataclass
class Ward:
    """クラスタリング後のward"""
    ward_id: int
    class_id: int
    class_name: str
    x: float  # 平均座標
    y: float
    frame_start: int
    frame_end: int
    detections: List[Detection] = field(default_factory=list)

    @property
    def confidence_avg(self) -> float:
        if not self.detections:
            return 0.0
        return sum(d.confidence for d in self.detections) / len(self.detections)

    @property
    def detection_count(self) -> int:
        return len(self.detections)

print("データ構造定義完了")

データ構造定義完了


## Step 1: YOLO推論（GPUバッチ対応）

In [5]:
#cell-5: 推論関数定義（GPUバッチ推論対応）

def run_inference(model: YOLO, match_dir: Path, conf: float = CONFIDENCE_THRESHOLD,
                  batch_size: int = BATCH_SIZE) -> List[Detection]:
    """
    1試合分の全フレームをバッチ推論し、検出結果を返す
    
    Args:
        model: YOLOモデル
        match_dir: 試合ディレクトリ
        conf: 信頼度閾値
        batch_size: GPUバッチサイズ
    """
    frame_dir = match_dir / "0"
    if not frame_dir.exists():
        print(f"フレームディレクトリが見つかりません: {frame_dir}")
        return []

    # フレーム一覧（ソート済み）
    frame_files = sorted(frame_dir.glob("*.png"), key=lambda p: int(p.stem))
    if not frame_files:
        print(f"フレームが見つかりません: {frame_dir}")
        return []

    detections: List[Detection] = []
    total_frames = len(frame_files)

    # バッチ推論
    with tqdm(total=total_frames, desc=f"推論中 ({match_dir.name})") as pbar:
        for i in range(0, total_frames, batch_size):
            # バッチ分のパスを取得
            batch_files = frame_files[i:i+batch_size]
            batch_paths = [str(p) for p in batch_files]
            frame_nums = [int(p.stem) for p in batch_files]

            # バッチ推論実行
            results = model(batch_paths, imgsz=IMAGE_SIZE, conf=conf, verbose=False)

            # 各フレームの結果を処理
            for frame_num, result in zip(frame_nums, results):
                for box in result.boxes:
                    class_id = int(box.cls[0])
                    class_name = model.names[class_id]
                    x, y, w, h = box.xywhn[0].tolist()
                    confidence = float(box.conf[0])

                    detections.append(Detection(
                        frame=frame_num,
                        class_id=class_id,
                        class_name=class_name,
                        x=x,
                        y=y,
                        w=w,
                        h=h,
                        confidence=confidence
                    ))

            pbar.update(len(batch_files))

    return detections


def save_raw_detections(detections: List[Detection], output_path: Path):
    """生の検出結果をCSVに保存"""
    output_path.parent.mkdir(parents=True, exist_ok=True)

    with open(output_path, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['frame', 'class_id', 'class_name', 'x', 'y', 'w', 'h', 'confidence'])

        for d in detections:
            writer.writerow([
                d.frame, d.class_id, d.class_name,
                f"{d.x:.6f}", f"{d.y:.6f}", f"{d.w:.6f}", f"{d.h:.6f}",
                f"{d.confidence:.4f}"
            ])

    print(f"生の検出結果を保存: {output_path} ({len(detections)}件)")

print(f"推論関数定義完了（バッチサイズ: {BATCH_SIZE}）")

推論関数定義完了（バッチサイズ: 8）


In [6]:
#cell-6: 1試合テスト推論（バッチ推論）

# 最初の試合でテスト
if DATASET_DIR.exists():
    match_dirs = sorted(DATASET_DIR.glob("JP1-*"))
    if match_dirs:
        test_match = match_dirs[0]
        print(f"テスト推論: {test_match.name}")
        print(f"バッチサイズ: {BATCH_SIZE}")
        
        # 最初の100フレームのみでテスト
        frame_dir = test_match / "0"
        frame_files = sorted(frame_dir.glob("*.png"), key=lambda p: int(p.stem))[:100]
        
        test_detections = []
        
        # バッチ推論
        start_time = time.time()
        
        for i in range(0, len(frame_files), BATCH_SIZE):
            batch_files = frame_files[i:i+BATCH_SIZE]
            batch_paths = [str(p) for p in batch_files]
            frame_nums = [int(p.stem) for p in batch_files]
            
            results = model(batch_paths, imgsz=IMAGE_SIZE, conf=CONFIDENCE_THRESHOLD, verbose=False)
            
            for frame_num, result in zip(frame_nums, results):
                for box in result.boxes:
                    class_id = int(box.cls[0])
                    class_name = model.names[class_id]
                    x, y, w, h = box.xywhn[0].tolist()
                    confidence = float(box.conf[0])
                    
                    test_detections.append(Detection(
                        frame=frame_num, class_id=class_id, class_name=class_name,
                        x=x, y=y, w=w, h=h, confidence=confidence
                    ))
        
        elapsed = time.time() - start_time
        fps = len(frame_files) / elapsed
        
        print(f"\n処理時間: {elapsed:.2f}秒")
        print(f"処理速度: {fps:.1f} FPS")
        print(f"検出数: {len(test_detections)}")
        
        # クラス別集計
        class_counts = defaultdict(int)
        for d in test_detections:
            class_counts[d.class_name] += 1
        print("\nクラス別:")
        for name, count in sorted(class_counts.items()):
            print(f"  {name}: {count}")

テスト推論: JP1-555620750
バッチサイズ: 8

処理時間: 5.40秒
処理速度: 18.5 FPS
検出数: 84

クラス別:
  stealth_ward: 9
  stealth_ward_enemy: 75


## Step 2: クラスタリング

In [7]:
#cell-7: クラスタリング関数定義

def distance(d1: Detection, d2: Detection) -> float:
    """2つの検出間のユークリッド距離（正規化座標）"""
    return np.sqrt((d1.x - d2.x) ** 2 + (d1.y - d2.y) ** 2)


def cluster_detections(detections: List[Detection],
                       distance_threshold: float = DISTANCE_THRESHOLD,
                       min_frames: int = MIN_FRAMES,
                       gap_tolerance: int = GAP_TOLERANCE) -> List[Ward]:
    """
    検出結果をクラスタリングしてward単位にまとめる
    """
    if not detections:
        return []

    # フレーム順にソート
    sorted_detections = sorted(detections, key=lambda d: d.frame)

    # アクティブなwardクラスタ（クラスごとに管理）
    active_wards: Dict[int, List[Ward]] = defaultdict(list)
    completed_wards: List[Ward] = []
    next_ward_id = 1

    for det in sorted_detections:
        matched = False

        # 同じクラスのアクティブなwardを検索
        for ward in active_wards[det.class_id]:
            if (distance(det, ward.detections[-1]) < distance_threshold and
                det.frame - ward.frame_end <= gap_tolerance):
                # 既存のwardに追加
                ward.detections.append(det)
                ward.frame_end = det.frame
                ward.x = sum(d.x for d in ward.detections) / len(ward.detections)
                ward.y = sum(d.y for d in ward.detections) / len(ward.detections)
                matched = True
                break

        if not matched:
            # 新規wardを作成
            new_ward = Ward(
                ward_id=next_ward_id,
                class_id=det.class_id,
                class_name=det.class_name,
                x=det.x,
                y=det.y,
                frame_start=det.frame,
                frame_end=det.frame,
                detections=[det]
            )
            active_wards[det.class_id].append(new_ward)
            next_ward_id += 1

        # 古いwardを完了リストに移動
        current_frame = det.frame
        for class_id in list(active_wards.keys()):
            still_active = []
            for ward in active_wards[class_id]:
                if current_frame - ward.frame_end > gap_tolerance:
                    completed_wards.append(ward)
                else:
                    still_active.append(ward)
            active_wards[class_id] = still_active

    # 残りのアクティブなwardを完了リストに追加
    for class_id in active_wards:
        completed_wards.extend(active_wards[class_id])

    # ノイズ除去
    filtered_wards = [w for w in completed_wards if w.detection_count >= min_frames]

    # ward_idを振り直し
    for i, ward in enumerate(sorted(filtered_wards, key=lambda w: w.frame_start), start=1):
        ward.ward_id = i

    return sorted(filtered_wards, key=lambda w: w.frame_start)


def save_wards(wards: List[Ward], output_path: Path):
    """クラスタリング後のward情報をCSVに保存"""
    output_path.parent.mkdir(parents=True, exist_ok=True)

    with open(output_path, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow([
            'ward_id', 'class_id', 'class_name',
            'x', 'y',
            'frame_start', 'frame_end',
            'detection_count', 'confidence_avg'
        ])

        for w in wards:
            writer.writerow([
                w.ward_id, w.class_id, w.class_name,
                f"{w.x:.6f}", f"{w.y:.6f}",
                w.frame_start, w.frame_end,
                w.detection_count, f"{w.confidence_avg:.4f}"
            ])

    print(f"ward情報を保存: {output_path} ({len(wards)}個のward)")

print("クラスタリング関数定義完了")

クラスタリング関数定義完了


In [8]:
#cell-8: クラスタリングテスト

if 'test_detections' in dir() and test_detections:
    test_wards = cluster_detections(test_detections)
    
    print(f"クラスタリング結果:")
    print(f"  検出数: {len(test_detections)} → ward数: {len(test_wards)}")
    
    if test_wards:
        # クラス別集計
        class_counts = defaultdict(int)
        for w in test_wards:
            class_counts[w.class_name] += 1
        print("\nクラス別ward数:")
        for name, count in sorted(class_counts.items()):
            print(f"  {name}: {count}")
        
        # 詳細表示
        print("\n先頭5件:")
        for w in test_wards[:5]:
            print(f"  ward_{w.ward_id}: {w.class_name} @ ({w.x:.3f}, {w.y:.3f}), frames {w.frame_start}-{w.frame_end}")

クラスタリング結果:
  検出数: 84 → ward数: 3

クラス別ward数:
  stealth_ward: 2
  stealth_ward_enemy: 1

先頭5件:
  ward_1: stealth_ward_enemy @ (0.436, 0.421), frames 18-99
  ward_2: stealth_ward @ (0.902, 0.828), frames 39-44
  ward_3: stealth_ward @ (0.903, 0.827), frames 77-79


## Step 3: 全試合バッチ処理

In [9]:
#cell-9: バッチ処理関数

def process_match(model: YOLO, match_dir: Path, conf: float = CONFIDENCE_THRESHOLD,
                  batch_size: int = BATCH_SIZE) -> Dict:
    """
    1試合を処理（推論 → クラスタリング → 保存）
    """
    print(f"\n{'='*60}")
    print(f"処理開始: {match_dir.name}")
    print(f"{'='*60}")

    # 1. 推論（バッチ対応）
    detections = run_inference(model, match_dir, conf=conf, batch_size=batch_size)

    if not detections:
        print("検出なし")
        return {"match": match_dir.name, "detections": 0, "wards": 0}

    # 2. 生の検出結果を保存
    raw_output = match_dir / "detections_raw.csv"
    save_raw_detections(detections, raw_output)

    # 3. クラスタリング
    wards = cluster_detections(detections)

    # 4. ward情報を保存
    wards_output = match_dir / "wards.csv"
    save_wards(wards, wards_output)

    # 5. 統計表示
    print(f"\n結果: 検出 {len(detections)} → ward {len(wards)}")

    return {"match": match_dir.name, "detections": len(detections), "wards": len(wards)}

print("バッチ処理関数定義完了")

バッチ処理関数定義完了


In [10]:
#cell-10: 全試合バッチ処理実行

# 処理対象の試合を取得
match_dirs = sorted(DATASET_DIR.glob("JP1-*"))
print(f"処理対象: {len(match_dirs)}試合")
print(f"バッチサイズ: {BATCH_SIZE}")

# 確認
print("\n処理を開始しますか？")
print("実行する場合は次のセルを実行してください")

処理対象: 123試合
バッチサイズ: 8

処理を開始しますか？
実行する場合は次のセルを実行してください


In [ ]:
#cell-11: バッチ処理実行（時間がかかります）

# 全試合を処理
start_time = time.time()
results = []
for match_dir in match_dirs:
    result = process_match(model, match_dir, conf=CONFIDENCE_THRESHOLD, batch_size=BATCH_SIZE)
    results.append(result)

total_time = time.time() - start_time

# サマリー
print("\n" + "="*60)
print("バッチ処理完了")
print("="*60)

total_detections = sum(r["detections"] for r in results)
total_wards = sum(r["wards"] for r in results)

print(f"処理試合数: {len(results)}")
print(f"総検出数: {total_detections}")
print(f"総ward数: {total_wards}")
print(f"総処理時間: {total_time/60:.1f}分")


処理開始: JP1-555620750


推論中 (JP1-555620750): 100%|██████████| 1585/1585 [00:34<00:00, 45.90it/s]


生の検出結果を保存: C:\dataset_20260105\JP1-555620750\detections_raw.csv (10010件)
ward情報を保存: C:\dataset_20260105\JP1-555620750\wards.csv (123個のward)

結果: 検出 10010 → ward 123

処理開始: JP1-555621265


推論中 (JP1-555621265): 100%|██████████| 2131/2131 [00:39<00:00, 53.94it/s]


生の検出結果を保存: C:\dataset_20260105\JP1-555621265\detections_raw.csv (11947件)
ward情報を保存: C:\dataset_20260105\JP1-555621265\wards.csv (163個のward)

結果: 検出 11947 → ward 163

処理開始: JP1-555622520


...

推論中 (JP1-555658734): 100%|██████████| 1001/1001 [00:20<00:00, 49.60it/s]


生の検出結果を保存: C:\dataset_20260105\JP1-555658734\detections_raw.csv (5127件)
ward情報を保存: C:\dataset_20260105\JP1-555658734\wards.csv (68個のward)

結果: 検出 5127 → ward 68

処理開始: JP1-555666735


推論中 (JP1-555666735): 100%|██████████| 1684/1684 [00:52<00:00, 32.16it/s]


生の検出結果を保存: C:\dataset_20260105\JP1-555666735\detections_raw.csv (11374件)
ward情報を保存: C:\dataset_20260105\JP1-555666735\wards.csv (150個のward)

結果: 検出 11374 → ward 150

処理開始: JP1-555667148


推論中 (JP1-555667148):  53%|█████▎    | 1024/1925 [00:31<00:37, 23.82it/s]

## Step 4: タイムライン統合

In [ ]:
#cell-12: WardTracker読み込み

from autoLeague.dataset.ward_tracker import WardTracker

print("WardTracker読み込み完了")

In [ ]:
#cell-13: タイムライン統合（1試合テスト）

# 最初の試合でテスト
if match_dirs:
    test_match = match_dirs[0]
    print(f"テスト: {test_match.name}")
    
    tracker = WardTracker(
        timeline_dir=TIMELINE_DIR,
        dataset_dir=DATASET_DIR,
        use_hungarian=True,  # ハンガリアン法（全体最適）を使用
    )
    
    tracker.process_match(test_match.name)
    
    # 結果確認
    matched_csv = test_match / "wards_matched.csv"
    if matched_csv.exists():
        df = pd.read_csv(matched_csv)
        print(f"\nwards_matched.csv:")
        print(f"  レコード数: {len(df)}")
        print(f"  カラム: {list(df.columns)}")
        display(df.head())

In [ ]:
#cell-14: タイムライン統合（全試合）

print(f"全{len(match_dirs)}試合のタイムライン統合を開始...")

tracker = WardTracker(
    timeline_dir=TIMELINE_DIR,
    dataset_dir=DATASET_DIR,
    use_hungarian=True,
)

for match_dir in tqdm(match_dirs, desc="タイムライン統合"):
    try:
        tracker.process_match(match_dir.name)
    except Exception as e:
        print(f"エラー ({match_dir.name}): {e}")

print("\nタイムライン統合完了")

## 結果確認

In [ ]:
#cell-15: 結果確認

print("=== 処理結果サマリー ===")

# 各試合のwards_matched.csvを確認
match_results = []
for match_dir in match_dirs:
    matched_csv = match_dir / "wards_matched.csv"
    if matched_csv.exists():
        df = pd.read_csv(matched_csv)
        match_results.append({
            "match": match_dir.name,
            "wards": len(df),
            "matched": len(df[df["match_status"] == "matched"]) if "match_status" in df.columns else 0,
        })

results_df = pd.DataFrame(match_results)
print(f"\n処理済み試合: {len(results_df)}")
print(f"総ward数: {results_df['wards'].sum()}")
print(f"マッチ済み: {results_df['matched'].sum()}")

if len(results_df) > 0:
    match_rate = results_df['matched'].sum() / results_df['wards'].sum() * 100
    print(f"マッチング率: {match_rate:.1f}%")

display(results_df)

In [ ]:
#cell-16: 次のステップ

print("=" * 60)
print("Phase 3 完了")
print("=" * 60)
print("\n次のステップ:")
print("  → 07_vision_score.ipynb でグリッド特徴量生成・モデル学習")